In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

In [ ]:
# Simulation parameters
N = 100                # number of particles
L = 10.0             # box size (0..L in both x and y)
dt = 0.05         # time step

# Initial state
pos=np.random.rand(N,2)*L
vel=np.random.uniform(-0.5,0.5,size=(N,2))
mass=1
radius = 0.1
plt.rcParams['animation.embed_limit'] = 200

k = 1
r0 = 0.2
rc = 2.0

In [ ]:
def pair_forces(pos):
    d = pos[None, :, :] - pos[:, None, :]
    d[:, :, 0] = d[:, :, 0] - L * np.round(d[:, :, 0] / L)
    d[:, :, 1] = d[:, :, 1] - L * np.round(d[:, :, 1] / L)
    r = np.linalg.norm(d, axis=2)
    mask = (r < rc) & (r > 1e-12)
    u = d / (r[:, :, None ] + 1e-12)
    fmag = -1 * k * (r - r0) * mask
    F = np.sum(fmag[:, :, None]*u, axis=1)
    return F

In [ ]:
def walls_reflect_other(pos, vel, L, x, y):
    vel[x, 0] *= -1.0
    pos[x, 0] = np.where(pos[x, 0] < 0, 0.0, np.where(pos[x, 0] > L, L, pos[x, 0]))

    vel[y, 1] *= -1.0
    pos[y, 1] = np.where(pos[y, 1] < 0, 0.0, np.where(pos[y, 1] > L, L, pos[y, 1]))

In [ ]:
def step_smooth(pos,vel,mass,dt,L,F_prev):
    if F_prev is None:
        F_prev = pair_forces(pos)
    a_prev = F_prev / mass
    vel = vel + 0.5 * a_prev * dt
    pos = pos + vel * dt
    x = (pos[:, 0] < 0) | (pos[:, 0] > L)
    y = (pos[:, 1] < 0) | (pos[:, 1] > L)
    walls_reflect_other(pos, vel, L, x, y)
    # new forces
    F_new = pair_forces(pos)
    a_new = F_new / mass
    vel = vel + 0.5 * a_new * dt
    return pos, vel, F_new


In [ ]:
def kinetic_energy(vel, mass):
     KE = 0.5 * mass * (np.sum(vel** 2))
     return KE

def potential_energy(pos, k=1.0, r0=0.8, rc=2.0):
    d = pos[None, :, :] - pos[:, None, :]       # pairwise displacement
    r = np.linalg.norm(d, axis=2)               # pairwise distance
    mask_valid = (r > 1e-12)                    # avoid self-interactions
    mask_inside = (r < rc) & mask_valid         # within cutoff
    U = -0.5 * k * (r - r0)**2 * mask_inside
    U += (-0.5 * k * (rc - r0)**2) * ((r >= rc) & mask_valid)
    # Divide by 2 to avoid double-counting pairs
    PE = np.sum(U) / 2.0
    return PE

In [ ]:
def detect_collisions(pos, radius):
    d = pos[None, :, :] - pos[:, None, :]
    r = np.linalg.norm(d, axis=2)
    colliding_pairs = np.array(np.where((r < 2 * radius) & (r > 1e-12))).T
    colliding_pairs = colliding_pairs[colliding_pairs[:,0] < colliding_pairs[:,1]]
    return colliding_pairs

def resolve_collision(pos, vel, i, j):
    r_rel = pos[i] - pos[j]

    dist = np.linalg.norm(r_rel)
    if dist == 0:
        return

    n = r_rel / dist

    v_i_n = np.dot(vel[i], n)
    v_j_n = np.dot(vel[j], n)


    vel[i] += (v_j_n - v_i_n) * n
    vel[j] += (v_i_n - v_j_n) * n

def step_with_collisions(pos, vel, mass, dt, L, F_prev, radius):
    pos, vel, F_new = step_smooth(pos, vel, mass, dt, L, F_prev)
    collisions = detect_collisions(pos, radius)
    for i, j in collisions:
        resolve_collision(pos, vel, i, j)
    return pos, vel, F_new


In [ ]:
plt.close('all')

fig, (ax, ax_energy) = plt.subplots(1, 2, figsize=(12, 6) , dpi =150)

ax.set_xlim(0, L)
ax.set_ylim(0, L)
ax.set_title("Particles(step_smooth)")
scatter = ax.scatter([], [], s=60, c='tab:blue', alpha=0.8)

ax_energy.set_xlabel("Time(Seconds)")
ax_energy.set_ylabel("Energy(Joules)")
ax_energy.set_title("Energy vs Time")

ke_line, = ax_energy.plot([], [], color='tab:red', label="KE")
pe_line, = ax_energy.plot([], [], color='tab:blue', label="PE")
te_line, = ax_energy.plot([], [], color='tab:green', label="TE")
ax_energy.legend(loc="upper left", bbox_to_anchor=(1.02, 1))
ke_times, ke_values, pe_values, te_values = [], [], [], []
delta_ke = ax_energy.text(0.02, 0.95, '', transform=ax_energy.transAxes, fontsize=12, color='blue')
ke_max_ss = 0
ke_count_ss = 0
ke_sum_ss = 0

F_prev = None

def init():
    scatter.set_offsets(pos)
    ke_line.set_data([], [])
    pe_line.set_data([], [])
    te_line.set_data([], [])
    ke_times.clear()
    ke_values.clear()
    pe_values.clear()
    te_values.clear()
    return scatter, ke_line, pe_line, te_line

delta_ke = ax_energy.text(0.02, 0.95, '', transform=ax_energy.transAxes, fontsize=12, color='blue')

def animate(frame):
    global pos, vel, F_prev, ke_max_ss, ke_sum_ss, ke_count_ss

    pos, vel, F_prev = step_with_collisions(pos, vel, mass, dt, L, F_prev,radius)

    scatter.set_offsets(pos)

    t = (frame+1)*dt
    KE = kinetic_energy(vel, mass)
    PE = potential_energy(pos, k, r0, rc)
    TE = KE + PE

    ke_times.append(t)
    ke_values.append(KE)
    pe_values.append(PE)
    te_values.append(TE)

    ke_line.set_data(ke_times, ke_values)
    pe_line.set_data(ke_times, pe_values)
    te_line.set_data(ke_times, te_values)

    ax_energy.set_xlim(0, max(dt, t))
    ymax = -PE*1.25
    ax_energy.set_ylim(-ymax, ymax)


    if len(te_values) > 1 and te_values[0] != 0:
        delta_te_percent = 100 * abs(te_values[-1] - te_values[-2]) / abs(te_values[-2])
        ke_max_ss = max(ke_max_ss, delta_te_percent)
        ke_sum_ss += delta_te_percent
        ke_count_ss += 1
        ke_mean = ke_sum_ss / ke_count_ss
        delta_ke.set_text(f"STD: TE {np.std(te_values):.2f}  STD: KE {np.std(ke_values):.2f}  STD: PE {np.std(pe_values):.2f} ")
    else:
        delta_ke.set_text("ΔTE%: 0.00%")

    return scatter, ke_line, pe_line, te_line, delta_ke

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=1000, interval=10, blit=True)

with open("2D_coll.html", "w") as f:
    f.write(anim.to_jshtml())
